### prueba scrapear!

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import time
import os

In [2]:
ua = UserAgent()
header = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

#### carrefour

In [3]:
list_pages_carrefour = list(range(0,336,24))
table_pages_carrefour=[]
for number in list_pages_carrefour:
    url_carrefour = f'https://www.carrefour.es/supermercado/bebidas/cerveza/todas-las-cervezas/N-1uq8b5u/c?No={number}'
    html = requests.get(url_carrefour, headers=header).text
    soup_carrefour = BeautifulSoup(html, 'lxml')
    table_carrefour = soup_carrefour.find_all({'article':'product-card-item'})
    table_pages_carrefour.append(table_carrefour)

In [4]:
table_pages_carrefour[0][23]

<article class="product-card-item">
<div class="right-side">
<div class="photo">
<a class="js-gap-product-click-super" data-product-discount="" data-productid="521028271" data-special-campaign="" href="/supermercado/cerveza-cruzcampo-pilsen-pack-de-24-botellas-de-25-cl-cruzcampo/R-521028271/p" title="Cerveza Cruzcampo Pilsen pack de 24 botellas de 25 cl.">
<img alt="Cerveza Cruzcampo Pilsen pack de 24 botellas de 25 cl." aria-describedby="product-521028271" sizes="(max-width: 280px) 80px,(max-width: 490px) 100px, (max-width: 768px) 560px, (max-width: 991px) 280px, (max-width: 1280px) 560px" src="https://static.carrefour.es/hd_280x_/img_pim_food/000147_00_1.jpg" srcset="https://static.carrefour.es/hd_560x_/img_pim_food/000147_00_1.jpg 560w, https://static.carrefour.es/hd_280x_/img_pim_food/000147_00_1.jpg 280w, https://static.carrefour.es/hd_100x_/img_pim_food/000147_00_1.jpg 100w, https://static.carrefour.es/hd_100x_/img_pim_food/000147_00_1.jpg  80w" width="100%"/>
</a>
</div>
</div>


In [5]:
len(table_pages_carrefour[0])

25

In [6]:
data_raw_carrefour=[]
for i in range(len(table_pages_carrefour)):
    for j in range(len(table_pages_carrefour[i])):
        rows_carrefour = table_pages_carrefour[i][j].find_all(['span','p','a'], {'class': ['price','price-less','format-price','js-gap-product-click-super']})
        promotion_carrefour = table_pages_carrefour[i][j].find_all(['p'], {'class': ['promocion-copy']})
        if len(rows_carrefour) != 0:
            if len(promotion_carrefour) != 0:
                data_raw_carrefour.append(rows_carrefour + promotion_carrefour)
            else:
                data_raw_carrefour.append(rows_carrefour)
                
            

In [7]:
data_raw_carrefour[20]

[<a class="js-gap-product-click-super" data-product-discount="0,20" data-productid="720007011" data-special-campaign="qvpromo730001_feliz-aniversario" href="/supermercado/cerveza-estrella-galicia-especial-pack-de-6-latas-de-25-cl-estrella-galicia/R-720007011/p" title="Cerveza Estrella Galicia especial pack de 6 latas de 25 cl.">
 <img alt="Cerveza Estrella Galicia especial pack de 6 latas de 25 cl." aria-describedby="product-720007011" sizes="(max-width: 280px) 80px,(max-width: 490px) 100px, (max-width: 768px) 560px, (max-width: 991px) 280px, (max-width: 1280px) 560px" src="https://static.carrefour.es/hd_280x_/img_pim_food/640535_00_1.jpg" srcset="https://static.carrefour.es/hd_560x_/img_pim_food/640535_00_1.jpg 560w, https://static.carrefour.es/hd_280x_/img_pim_food/640535_00_1.jpg 280w, https://static.carrefour.es/hd_100x_/img_pim_food/640535_00_1.jpg 100w, https://static.carrefour.es/hd_100x_/img_pim_food/640535_00_1.jpg  80w" width="100%"/>
 </a>,
 <span class="price-less">
 3,00 €

In [8]:
len(data_raw_carrefour)

334

In [9]:
#price in euros
# price_raw = data_raw[0][1].text

def get_price(data_text):
    price_raw = data_text.text
    price = re.sub('\xa0€','',price_raw)
    price = re.sub('\n','',price)
    return float(re.sub(",",'.',price))

In [10]:
#price per liter
# price_l_raw = data_raw[0][2].text

def get_price_l(data_text):
    price_l_raw = data_text.text
    price_l = re.findall('[0-9]+,[0-9]+', price_l_raw)
    price_liter = re.sub(',','.',price_l[0])
    return float(price_liter)

In [11]:
#title
# title_raw = data_raw[0][3].text

def get_title(data_text):
    title_raw = data_text.text
    title = re.sub("\n",'',title_raw)
    return title

In [12]:
#promotion product

def get_promotion(data_text):
        promotion_raw = data_text.text
        promotion = re.sub("\n",'',promotion_raw)
        return promotion


In [13]:
#brand
def get_brand(data_text):
    brand_raw = data_text.text
    brand = re.split('(Cerveza )|( pack)|( botella)|( lata)+|(cl)',brand_raw)
    return brand[6]

In [14]:
#container (carrefour  / alcampo)
def get_container(data_text):
    container_raw = data_text.text
    container = re.findall('botella|lata|barril',container_raw)
    if len(container)!=0:
        return container[0]
    else:
        return 'No specified'

In [15]:
#volumen
def get_volumen_unid(data_text):
    volumen_raw = data_text.text 
    volumen = re.findall('[0-9]+ cl',volumen_raw)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9]+[ mcl]+',volumen_raw)
        return volumen_l[0]

In [16]:
#quantity (carrefour  / alcampo)
def get_quantity_pack(data_text):
    quantity_raw = data_text.text
    quantity_split = re.split('pack de|pack',quantity_raw)
    if len(quantity_split)==2:
        quantity = re.findall('[0-9]+',quantity_split[1])
        return quantity[0]
    else:
        return 1

In [17]:
#image
def get_image(data_text):
    return data_text.find_all('img')[0].get('src')

In [18]:
data_beer_carrefour = pd.DataFrame(index=range(0,len(data_raw_carrefour)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_carrefour)):
    data_beer_carrefour.iloc[beer_number,0] = get_price(data_raw_carrefour[beer_number][1])
    data_beer_carrefour.iloc[beer_number,1] = get_price_l(data_raw_carrefour[beer_number][2])
    data_beer_carrefour.iloc[beer_number,2] = get_title(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,4] = get_brand(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,5] = get_container(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,6] = get_volumen_unid(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,7] = get_quantity_pack(data_raw_carrefour[beer_number][3])
    data_beer_carrefour.iloc[beer_number,8] = get_image(data_raw_carrefour[beer_number][0])
    data_beer_carrefour.iloc[beer_number,9] = 'Carrefour'
    if len(data_raw_carrefour[beer_number]) > 4:
        data_beer_carrefour.iloc[beer_number,3] = get_promotion(data_raw_carrefour[beer_number][4])
    else:
        data_beer_carrefour.iloc[beer_number,3] = 'No promotion'


In [19]:
data_beer_carrefour.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,11.21,1.89,Cerveza Heineken Lager pack de 18 latas de 33 cl.,No promotion,Heineken Lager,lata,33 cl,18,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
1,14.8,1.6,Cerveza Cruzcampo Pilsen pack de 28 latas de 3...,No promotion,Cruzcampo Pilsen,lata,33 cl,28,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
2,5.15,1.56,Cerveza El Aguila pack de 10 latas de 33 cl.,No promotion,El Aguila,lata,33 cl,10,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
3,0.25,0.76,Cerveza Carrefour Holandesa lata 33 cl.,No promotion,Carrefour Holandesa,lata,33 cl,1,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour
4,14.15,1.53,Cerveza Mahou Clásica pack de 28 latas de 33 cl.,No promotion,Mahou Clásica,lata,33 cl,28,https://static.carrefour.es/hd_280x_/img_pim_f...,Carrefour


In [20]:
sum(data_beer_carrefour['price'])

913.0800000000002

## Alcampo

In [21]:
list_pages_alcampo = list(range(0,10))
table_pages_alcampo=[]
for number in list_pages_alcampo:
    url_alcampo = f'https://www.alcampo.es/compra-online/bebidas/cervezas/c/W1107?q=%3Arelevance&page={number}'
    html = requests.get(url_alcampo, headers=header).text
    soup_alcampo = BeautifulSoup(html, 'lxml')
    table_alcampo = soup_alcampo.find_all('div',{'class':'productGridItem'})
    table_pages_alcampo.append(table_alcampo)
    

In [22]:
len(table_pages_alcampo[0])

48

In [23]:
len(table_pages_alcampo)

10

In [24]:
data_raw_alcampo=[]
for i in range(len(table_pages_alcampo)):
    for j in range(len(table_pages_alcampo[i])):
        rows_alcampo = table_pages_alcampo[i][j].find_all(['span','div'],{'class': ['price',
                                                                                    'productName',
                                                                                    'thumb cut-alt-img']})
        promotion_alcampo = table_pages_alcampo[i][j].find_all('div', {'class': 'financiacionMensual'})
        if len(rows_alcampo) != 0:
            if len(promotion_alcampo) != 0:
                data_raw_alcampo.append(rows_alcampo + promotion_alcampo)
            else:
                data_raw_alcampo.append(rows_alcampo)
                

In [25]:
len(data_raw_alcampo)

432

In [26]:
#price alcampo / corte ingles / Dia
def get_price_alcampo(data_text):
    price_raw = data_text.text
    price = re.findall('[0-9,]+',price_raw)
    return float(re.sub(",",'.',price[0]))


In [27]:
#price_liter alcampo / corte ingles / Dia
def get_price_liter_alcampo(data_text):
    price_raw = data_text.text
    price = re.findall('[0-9,]+',price_raw)
    if len(price)==1:
        return float(re.sub(",",'.',price[0]))
    else:
        return float(re.sub(",",'.',price[1]))


In [28]:
#title alcampo / corte ingles
def get_tittle_alcampo(data_text):
    tittle_raw = data_text.text
    tittle = re.sub('\n','',tittle_raw)
    return tittle


In [29]:
#promotion alcampo
def get_promotion_alcampo(data_text):
    promotion_raw = data_text.text
    promotion = re.sub('\r\n\r\n\t\t\t\t\t\t\t\t\t\t','',promotion_raw)
    return promotion

In [342]:
#brand alcampo
def get_brand_alcampo(data_text):
    brand_raw = data_text.text
    brand = re.sub('\n','',brand_raw)
    brand_2 = re.split('0,0',brand)
    if len(brand_2)==1:
        brand_3 = re.findall("[A-Z 0-9,À-ÿ&'-]{3,} ",brand_2[0])
        return brand_3[0]
    else:
        brand_3 = re.findall("[A-Z 0-9,À-ÿ&'-]{3,} ",brand_2[1])
        return brand_3[0]

In [31]:
#volumen alcampo
def get_volumen_unid_alcampo(data_text):
    volumen_raw = data_text.text 
    volumen = re.findall('[0-9.,]+ cl',volumen_raw)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9.,]+[ mclL]+',volumen_raw)
        if len(volumen_l)==2:
            return volumen_l[1]
        else:
            return volumen_l[0]

In [32]:
#image alcampo
def get_image_alcampo(data_text):
    return data_text.find_all('img')[0].get('data-blzsrc')

In [338]:
data_raw_alcampo[338][1].text

'\n\nCerveza Alemana de trigo con toque cítricos MAISEL & FRIENDS WEIZEN IPA botella 33 cl. \n\n'

In [343]:
data_beer_alcampo= pd.DataFrame(index=range(0,len(data_raw_alcampo)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_alcampo)):
    data_beer_alcampo.iloc[beer_number,0] = get_price_alcampo(data_raw_alcampo[beer_number][2])
    data_beer_alcampo.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_alcampo[beer_number][2])
    data_beer_alcampo.iloc[beer_number,2] = get_tittle_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,4] = get_brand_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,5] = get_container(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,7] = get_quantity_pack(data_raw_alcampo[beer_number][1])
    data_beer_alcampo.iloc[beer_number,8] = get_image_alcampo(data_raw_alcampo[beer_number][0])
    data_beer_alcampo.iloc[beer_number,9] = 'Alcampo'
    if len(data_raw_alcampo[beer_number]) > 3:
        data_beer_alcampo.iloc[beer_number,3] = get_promotion_alcampo(data_raw_alcampo[beer_number][3])
    else:
        data_beer_alcampo.iloc[beer_number,3] = 'No promotion'

In [346]:
data_beer_alcampo.head(339)

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,11.99,1.29,Cerveza Clásica MAHOU pack de 28 latas de 33 cl.,oferta válida hasta el 14/10/2020,MAHOU,lata,33 cl,28,https://www.alcampo.es/media/ha9/h39/100335357...,Alcampo
1,13.49,1.45,Cervezas MAHOU 5 ESTRELLAS pack 28 latas x 33 cl.,No promotion,MAHOU 5 ESTRELLAS,lata,33 cl,28,https://www.alcampo.es/media/h11/h90/902116140...,Alcampo
2,6.66,1.68,Cerveza HEINEKEN pack de 12 latas de 33 centi...,No promotion,HEINEKEN,lata,33 c,12,https://www.alcampo.es/media/h67/h4f/954684103...,Alcampo
3,5.28,1.33,"Cervezas AMSTEL 100 % MALTA pack 12 uds, de 3...",No promotion,AMSTEL 100,No specified,33 cl,12,https://www.alcampo.es/media/hc9/h32/104726094...,Alcampo
4,5.62,1.41,Cervezas Clásica MAHOU pack de 12 latas de 33...,No promotion,MAHOU,lata,33 c,12,https://www.alcampo.es/media/h64/hc7/965932407...,Alcampo
...,...,...,...,...,...,...,...,...,...,...
334,1.27,3.84,Cerveza con zumo natural de limón ARRIACA RADL...,No promotion,ARRIACA RADLER 33,No specified,33 cl,1,https://www.alcampo.es/media/h33/h0e/104286000...,Alcampo
335,3.99,2.01,Cerveza negra portuguesa SUPER BOCK pack de 6 ...,No promotion,SUPER BOCK,botella,33 c,6,https://www.alcampo.es/media/h17/hff/889006974...,Alcampo
336,1.73,5.24,Cerveza escocesa de tipo ale fuerte INNIS&GUNN...,No promotion,INNIS&GUNN ORIGINAL,botella,33 cl,1,https://www.alcampo.es/media/h68/h98/896017543...,Alcampo
337,1.35,4.09,Cerveza ARRIACA RUBIA 33 cl.,No promotion,ARRIACA RUBIA 33,No specified,33 cl,1,https://www.alcampo.es/media/hc1/hf3/104147254...,Alcampo


## El corte ingles

In [37]:
list_pages_corteingles = list(range(1,26))
table_pages_corteingles =[]
for number in list_pages_corteingles:
    url_corteingles = f'https://elcorteingles.es/supermercado/bebidas/cervezas/{number}/'
    html = requests.get(url_corteingles, headers=header).text
    soup_corteingles= BeautifulSoup(html, 'lxml')
    table_corteingles = soup_corteingles.find_all('div',{'class':'grid-item'})
    table_pages_corteingles.append(table_corteingles)

In [170]:
len(table_pages_corteingles)

25

In [171]:
len(table_pages_corteingles[24])

24

In [294]:
data_raw_corteingles=[]
for i in range(len(table_pages_corteingles)):
    for j in range(len(table_pages_corteingles[i])):
        rows_corteingles = table_pages_corteingles[i][j].find_all(['div','h3'],{'class': ['prices-price',
                                                                                    'product_tile-description',
                                                                                    'product_tile-image _fade']})
        promotion_corteingles = table_pages_corteingles[i][j].find_all('div', {'class': 'product_tile-offer offer'})
        if len(rows_corteingles) > 3:
            if len(promotion_corteingles) != 0:
                data_raw_corteingles.append(rows_corteingles + promotion_corteingles)
            else:
                data_raw_corteingles.append(rows_corteingles)
                

In [295]:
len(data_raw_corteingles)

448

In [296]:
for beer in data_raw_corteingles: 
    verification = re.split('cervez|cider|cl',beer[3].text)
    if len(verification)==1:
        beer.pop(1)

In [46]:
#brand corte ingles
def get_brand_corteingles(data_text):
    brand_raw = data_text.text
    brand = re.split('cervez',brand_raw)
    return brand[0]

In [47]:
#volumen corte ingles
def get_volumen_corteingles(data_text):
    brand_raw = data_text.text
    brand = re.split('botel',brand_raw)
    return brand[0]

In [48]:
#image corte ingles
def get_image_corteingles(data_text):
    return 'https:'+data_text.find_all('img')[0].get('src')

In [291]:
data_raw_corteingles[321][3].text

'LA SAGRA Blue Moon cerveza turbia artesana castellana botella 33 cl '

In [297]:
data_beer_corteingles = pd.DataFrame(index=range(0,len(data_raw_corteingles)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_corteingles)):
    data_beer_corteingles.iloc[beer_number,0] = get_price_alcampo(data_raw_corteingles[beer_number][1])
    data_beer_corteingles.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_corteingles[beer_number][2])
    data_beer_corteingles.iloc[beer_number,2] = get_tittle_alcampo(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,4] = get_brand_corteingles(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,5] = get_container(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,7] = get_quantity_pack(data_raw_corteingles[beer_number][3])
    data_beer_corteingles.iloc[beer_number,8] = get_image_corteingles(data_raw_corteingles[beer_number][0])
    data_beer_corteingles.iloc[beer_number,9] = 'El Corte Ingles'
    if len(data_raw_corteingles[beer_number]) > 4:
        data_beer_corteingles.iloc[beer_number,3] = get_promotion_alcampo(data_raw_corteingles[beer_number][4])
        if len(data_raw_corteingles[beer_number]) > 5:
            data_beer_corteingles.iloc[beer_number,3] += '..SECOND OFFERT: '+ get_promotion_alcampo(data_raw_corteingles[beer_number][5])
    else:
        data_beer_corteingles.iloc[beer_number,3] = 'No promotion'    

In [317]:
data_beer_corteingles.head(150)

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,6.72,1.7,MAHOU CLASICA cerveza rubia original pack 12 l...,21% de descuento por compras superiores a 20 €...,MAHOU CLASICA,lata,33 cl,12,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles
1,14.84,1.61,MAHOU CLASICA cerveza rubia original pack 28 l...,2ª-50%2ª unidad al 50% de descuentoPuedes comb...,MAHOU CLASICA,lata,33 cl,28,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles
2,5,1.67,MAHOU CLASICA cerveza rubia original pack 12 b...,2ª-50%2ª unidad al 50% de descuentoPuedes comb...,MAHOU CLASICA,botella,25 cl,12,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles
3,1.37,1.37,MAHOU CLASICA cerveza rubia original botella 1 l,21% de descuento por compras superiores a 20 €...,MAHOU CLASICA,botella,1 l,1,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles
4,2.68,1.79,MAHOU CLASICA cerveza rubia original pack 6 bo...,21% de descuento por compras superiores a 20 €...,MAHOU CLASICA,botella,25 cl,6,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles
...,...,...,...,...,...,...,...,...,...,...
145,4.15,27.67,AMBAR EXPORT cerveza rubia nacional extra fuer...,21% de descuento por compras superiores a 20 €...,AMBAR EXPORT,botella,25 cl,6,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles
146,1.05,3.18,AMBAR EXPORT cerveza rubia Red Brew 3 maltas b...,6x5Lleva 6 y paga 5..SECOND OFFERT: 21% de des...,AMBAR EXPORT,botella,33 cl,1,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles
147,3.59,2.99,MORITZ cerveza rubia nacional botella Pack 6 b...,21% de descuento por compras superiores a 20 €...,MORITZ,botella,20 cl,1,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles
148,1.09,3.3,MORITZ cerveza rubia nacional botella 33 cl,21% de descuento por compras superiores a 20 €...,MORITZ,botella,33 cl,1,https://cdn.grupoelcorteingles.es/SGFM/dctm/ME...,El Corte Ingles


## DIA

In [52]:
list_pages_dia= list(range(0,3))
table_pages_dia =[]
for number in list_pages_dia:
    url_dia = f'https://www.dia.es/compra-online/productos/bebidas/cervezas/c/WEB.008.064.00000?page={number}&disp='
    html = requests.get(url_dia, headers=header).text
    soup_dia= BeautifulSoup(html, 'lxml')
    table_dia = soup_dia.find_all('div',{'class':'prod_grid'})
    table_pages_dia.append(table_dia)

In [53]:
len(table_pages_dia)

3

In [54]:
len(table_pages_dia[2])

21

In [55]:
data_raw_dia=[]
for i in range(len(table_pages_dia)):
    for j in range(len(table_pages_dia[i])):
        rows_dia = table_pages_dia[i][j].find_all(['div','span'],{'class': ['price_container','details',
                                                                            'thumb']})
        if len(rows_dia) > 2:
            data_raw_dia.append(rows_dia)
                

In [56]:
len(data_raw_dia)

68

In [57]:
#title Dia
def get_tittle_dia(data_text):
    tittle_raw = data_text.text
    tittle = re.sub('\r\n\t\t\t\t','',tittle_raw)
    return tittle

In [58]:
#brand Dia
def get_brand_dia(data_text):
    brand_raw = data_text.text
    brand_sub = re.sub('\r\n\t\t\t\t','',brand_raw)
    brand = re.split('cervez',brand_sub)
    return brand[0]

In [59]:
#image dia
def get_image_dia(data_text):
    return data_text.find_all('img')[0].get('data-original')

In [60]:
data_raw_dia[24][2].text

'\n\r\n\t\t\t\t\t\r\n\t\t\t\t\t0,35\xa0€\n(1,06\xa0€/l.)\n'

In [61]:
data_beer_dia= pd.DataFrame(index=range(0,len(data_raw_dia)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_dia)):
    data_beer_dia.iloc[beer_number,0] = get_price_alcampo(data_raw_dia[beer_number][2])
    data_beer_dia.iloc[beer_number,1] = get_price_liter_alcampo(data_raw_dia[beer_number][2])
    data_beer_dia.iloc[beer_number,2] = get_tittle_dia(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,4] = get_brand_dia(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,5] = get_container(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,6] = get_volumen_unid_alcampo(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,7] = get_quantity_pack(data_raw_dia[beer_number][1])
    data_beer_dia.iloc[beer_number,8] = get_image_dia(data_raw_dia[beer_number][0])
    data_beer_dia.iloc[beer_number,9] = 'Dia'
    if len(data_raw_dia[beer_number]) > 3:
        data_beer_dia.iloc[beer_number,3] = get_promotion_alcampo(data_raw_dia[beer_number][3])
    else:
        data_beer_dia.iloc[beer_number,3] = 'No promotion' 

In [315]:
data_beer_dia.head()

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,0.7,0.7,RAMBLERS cerveza lager botella 1 lt,No promotion,RAMBLERS,botella,1 l,1,https://s0.dia.es/medias/h23/h3e/1023867263388...,Dia
1,0.43,1.3,AMSTEL cerveza lata 33 cl,No promotion,AMSTEL,lata,33 cl,1,https://s3.dia.es/medias/ha3/hd0/9847080747038...,Dia
2,6.61,2.2,MAHOU 5 ESTRELLAS cerveza pack 12 botellas 25 cl,No promotion,MAHOU 5 ESTRELLAS,botella,25 cl,12,https://s3.dia.es/medias/ha0/hb5/9847978459166...,Dia
3,0.45,0.9,DIA cerveza alemana lata 50 cl,No promotion,DIA,lata,50 cl,1,https://s0.dia.es/medias/h29/h37/8821852569630...,Dia
4,7.08,1.79,MAHOU 5 ESTRELLAS cerveza pack 12 latas 33 cl,No promotion,MAHOU 5 ESTRELLAS,lata,33 cl,12,https://s3.dia.es/medias/heb/h09/9847113449502...,Dia


## Eroski

In [129]:
os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}'  
os.environ['PATH']

driver_options = Options()
driver_options.headless = True
driver = webdriver.Firefox(options=driver_options)

driver.get('https://supermercado.eroski.es/es/supermercado/2060211-bebidas/2060233-cervezas/') 
for i in range(20):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1.1)
    
selenium_raw_eroski = driver.find_elements_by_class_name('product-item')

In [130]:
len(selenium_raw_eroski)

240

In [131]:
data_raw_eroski = []
for i in selenium_raw_eroski:
    data_raw_eroski.append(i.text)


In [132]:
#price eroski

def get_price_eroski(data_text):
    price = re.findall('[0-9,]+',data_text)
    last = len(price)-1
    return float(re.sub(",",'.',price[last]))

In [133]:
#price_liter eroski

def get_price_liter_eroski(data_text):
    price_raw = re.split(' LITRO A ',data_text)
    if len(price_raw)>1:
        price = re.findall('[0-9,]+',price_raw[1])
    else:
        price = re.findall('[0-9,]{2,}',data_text)
    return float(re.sub(",",'.',price[0]))

In [134]:
#title eroski
def get_tittle_eroski(data_text):
    tittle = re.split('\n',data_text)
    return tittle[1]

In [358]:
#brand eroski
def get_brand_eroski(data_text):
    brand_raw = re.split('erveza |, |ider ',data_text)
    return brand_raw[1]
#     if len(brand_raw)>1:
#         brand = re.findall('[A-Z 0-9À-ÿ-]{3,}',brand_raw[1])
#         return brand[0]
#     else:
#         brand = re.findall('[A-Z 0-9À-ÿ-]{3,}',data_text)
#         return brand[1]


In [136]:
#container eroski
def get_container_eroski(data_text):
    container = re.findall('botella|lata|barril',data_text)
    if len(container)!=0:
        return container[0]
    else:
        return 'No specified'

In [137]:
#volumen eroski
def get_volumen_unid_eroski(data_text):
    volumen = re.findall('[0-9.,]+ cl',data_text)
    if len(volumen)!=0:
        return volumen[0]
    else:
        volumen_l = re.findall('[0-9.,]+[ mclL]+',data_text)
        if len(volumen_l)==2:
            return volumen_l[1]
        else:
            return volumen_l[0]

In [138]:
#quantity eroski
def get_quantity_pack_eroski(data_text):
    quantity_split = re.split('pack de|pack',data_text)
    if len(quantity_split)==2:
        quantity = re.findall('[0-9]+',quantity_split[1])
        return quantity[0]
    else:
        return 1

In [139]:
#promotion eroski

def get_promotion_eroski(data_text):
        promotion = re.split("\n",data_text)
        if len(promotion)>8:
            number = len(promotion)-6
            return promotion[number] + ' ' + promotion[number+1]
        else:
            return 'No promotion'


In [310]:
data_raw_eroski[198]

'  0\nCerveza VOLL-DAMM, botellín 66 cl\n1 LITRO A 2,42 €\nAhora\n1,60 €\nAñadir\nAñadir a la lista'

In [141]:
data_raw_eroski[3]

'  0\nCerveza MAHOU 5 Estrellas, pack 24x33 cl\n1 LITRO A 0,97 €\n13,59 €\nAntes\nAhora\n7,75 €\nAñadir\nAñadir a la lista'

In [359]:
data_beer_eroski= pd.DataFrame(index=range(0,len(data_raw_eroski)),columns=['price','price_liter',
                                                                           'title','promotion','brand',
                                                                          'container','volumen_unid',
                                                                          'quantity_pack','image_url',
                                                                        'supermarket'])
for beer_number in range(len(data_raw_eroski)):
    data_beer_eroski.iloc[beer_number,0] = get_price_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,1] = get_price_liter_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,2] = get_tittle_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,3] = get_promotion_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,4] = get_brand_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,5] = get_container_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,6] = get_volumen_unid_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,7] = get_quantity_pack_eroski(data_raw_eroski[beer_number])
    data_beer_eroski.iloc[beer_number,8] = 'Not image'
    data_beer_eroski.iloc[beer_number,9] = 'Eroski'

In [360]:
data_beer_eroski.head(200)

,price,price_liter,title,promotion,brand,container,volumen_unid,quantity_pack,image_url,supermarket
0,12.59,1.58,"Cerveza SAN MIGUEL, pack lata 24x33 cl",No promotion,SAN MIGUEL,lata,33 cl,24,Not image,Eroski
1,6.47,1.64,"Cerveza SAN MIGUEL, pack lata 12x33 cl",No promotion,SAN MIGUEL,lata,33 cl,12,Not image,Eroski
2,6.72,1.7,"Cerveza MAHOU Clásica, pack 12x33 cl",No promotion,MAHOU Clásica,No specified,33 cl,12,Not image,Eroski
3,7.75,0.97,"Cerveza MAHOU 5 Estrellas, pack 24x33 cl","13,59 € Antes",MAHOU 5 Estrellas,No specified,33 cl,24,Not image,Eroski
4,5.31,1.76,"Cerveza SAN MIGUEL, pack botellín 12x25 cl",2ª unidad -50%,SAN MIGUEL,No specified,25 cl,12,Not image,Eroski
...,...,...,...,...,...,...,...,...,...,...
195,0.99,3.96,"Cider sabor frutos rojos LADRÓN DE MANZANAS, b...",2ª unidad -50%,sabor frutos rojos LADRÓN DE MANZANAS,No specified,25 cl,1,Not image,Eroski
196,0.22,0.67,"Cerveza CORGON GARD, lata 33 cl",No promotion,CORGON GARD,lata,33 cl,1,Not image,Eroski
197,0.68,2.06,"Cerveza extra AMSTEL, lata 33 cl",No promotion,extra AMSTEL,lata,33 cl,1,Not image,Eroski
198,1.6,2.42,"Cerveza VOLL-DAMM, botellín 66 cl",No promotion,VOLL-DAMM,No specified,66 cl,1,Not image,Eroski


In [363]:
def save_df_products(data,name):
    data.to_csv(f'../data/processed/data_beer_{name}.csv', index=False)

In [364]:
save_df_products(data_beer_carrefour, 'carrefour')
save_df_products(data_beer_alcampo, 'alcampo')
save_df_products(data_beer_corteingles, 'corteingles')
save_df_products(data_beer_dia,'dia')
save_df_products(data_beer_eroski,'eroski')